## Classificação de números através de imagens
O algorimo tem como objetivo desenvolver um modelo que consiga através de imagens contendo números escrito a mão, classificar qual o número. Para isso foi usado o dataset MNIST, que é encontrado nos datasets da lib Keras. A base de dados contém 60000 imagens de escrita a mão dos números de 0 a 9. 

Para a resolução do problema foi escolhido o uso de __Redes Neurais Artificiais__, ao final do processo será possível fornecer imagens ao modelo e o mesmo classificará qual dígito de 0 a 9 a imagem contém.

##### 1. Bibliotecas usadas

Foram usadas bibliotecas padrões para manipulação de dados como __numpy__, para o desenvolvimento do modelo e algumas ferramentas extras relacionadas foi usado o __keras e sklearn__.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from keras.datasets import mnist
from keras.callbacks import ModelCheckpoint

from skimage import color
from skimage import io

##### 2. Funções utilitárias

Foram definidas algumas funções básicas para ajudar no desenvolvimento.

__describe_image__: Exibe a imagem em formato grayscale.<br>
__transform_image__: Transforma uma imagem no formato de matrix para um vetor.

In [ ]:
def describe_image(image):
    plt.imshow(image, cmap='gray')

def transform_image_array_to_model(images_array):
    transformed_data = images_array.copy()
    transformed_data = transformed_data.reshape(len(transformed_data), np.prod(transformed_data.shape[1:]))
    transformed_data = transformed_data / 255
    return transformed_data.astype('float32')

def transform_image_class_to_model(images_class):
    transformed_target = images_class.copy()
    transformed_target = pd.get_dummies(transformed_target)
    return transformed_target.astype('float32')

##### 3. Carregamento dos dados

O acesso ao dataset acontece através da lib do __keras__, onde conseguimos importar no módulo de datasets a base MNIST. Ao carregar os dados já é possível obter a divisão entre dados de treino e de teste, além de já estarem normalizados para o uso.

In [ ]:
(training_data, training_target), (test_data, test_target) = mnist.load_data()

print(training_data.shape, test_data.shape, training_target.shape, test_target.shape)

##### 4. Visualizando e ententendo as imagens

Percebe-se que os dados relativos a cada imagem são matrizes no formato __28 por 28__, contendo um valor na escala gray. Só a nível de curiosidade, vamos visualizar um item dos dados.

In [ ]:
describe_image(training_data[0])

##### 5. Transformação das imagens

Será gerado nesse passo os previsores e as classes que serão usados para o treino e teste do modelo. O previsores (imagens) que a princípio são __matrizes 28x28__, serão transformados em um array de __784 posições__... já as classes, são valores numéricos de __0-9__, informando qual o tipo de número presente naquela imagem, essas classes serão transformadas em one hot encoder, gerando 10 colunas para cada atributo.

In [ ]:
transformed_training_data = transform_image_array_to_model(training_data)
transformed_test_data = transform_image_array_to_model(test_data)

transformed_training_target = transform_image_class_to_model(training_target)
transformed_test_target = transform_image_class_to_model(test_target)

print(transformed_training_data.shape, transformed_test_data.shape, transformed_training_target.shape, transformed_test_target.shape)

##### 6. Criação do modelo

Criou-se o modelo, definido em 4 camadas incluindo a camada de saída, a camada de entrada contém 784 perceptrons, pois cada imagem foi mapeada em um array de 784 colunas. Adicionamos dropout para evitar o overfitting, e usou-se a função de loss do tipo __categorical_crossentropy__.

A camada de saída apresenta 10 perceptrons devido a saída ser 10 classes, indo de 0 até 9, e as classes foram mapeadas em one hot encoder, gerando assim 10 colunas de saída.

In [ ]:
model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=784))
model.add(Dropout(0.2))

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

##### 7. Variáveis e treinamento

Definimos que o modelo iria realizar __100 epochs__, visto que rapidamente já conseguimos atingir uma boa precisão. Adicionamos também um _check pointer_ para salvar o melhor modelo entre os 100 ciclos. Em seguida, o modelo será treinado.

In [ ]:
num_epochs = 100

checkpointer = ModelCheckpoint(filepath='saved_models/image_classification.hdf5', 
                               save_best_only=True)

model.fit(transformed_training_data, transformed_training_target, 
          epochs=num_epochs, 
          validation_data=(transformed_test_data, transformed_test_target),
          callbacks=[checkpointer])

##### 8. Teste de precisão

Ao fim do treinamento, rodamos o modelo com os dados separados de teste para fazer uma avaliação da sua performace.

In [ ]:
accuracy = model.evaluate(transformed_test_data, transformed_test_target, verbose = 0)
print('Precisão do modelo é de ' + str(accuracy[1]*100) + '%')

##### 9. Teste manual

Na pasta __images_extras__ foi adicionado alguns exemplos de imagens feitos por mim, para testar a eficiência do modelo. Você também pode tentar, basta fazer uma imagem com fundo preto o número na cor branca, lembrando que a imagem deve ser no formato __28x28__. Vale salientar que o modelo foi treinado com dados limpos e padronizados do dataset MNIST, então caso você coloque uma imagem muito diferente do padrão que o modelo foi treinado, possivelmente ele não será eficiente, nesse algoritmo estamos partindo do pressuposto que os dados já estão padronizados e tratados.

In [ ]:
def load_png_images(path):
    image_files = glob(path)
    images_processed = np.array([color.rgb2gray(io.imread(image)) for image in image_files])
    return image_files, images_processed

image_files, images_processed = load_png_images('images_extras/*.png')
transformed_images_to_model = transform_image_array_to_model(images_processed * 255)

manual_test_result = model.predict(transformed_images_to_model)
manual_test_result = np.where(manual_test_result > 0.5, 1, 0)

for image_index in range(0, len(images_processed)):
    print('A imagem "' + str(image_files[image_index]) + '" foi classificado como o número ' + str(np.argmax(manual_test_result[image_index])))